In [1]:
options(warn=2, error=recover)
install_if_missing <- function(pkg, github = NULL) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    message(sprintf("Installing package: %s", pkg))
    tryCatch({
      if (is.null(github)) {
        install.packages(pkg, repos = "https://cloud.r-project.org")
      } else {
        if (!requireNamespace("remotes", quietly = TRUE)) {
          install.packages("remotes", repos = "https://cloud.r-project.org")
        }
        remotes::install_github(github)
      }
    }, error = function(e) {
      message(sprintf("Failed to install %s: %s", pkg, e$message))
      return(NULL)
    })
  }
  if (requireNamespace(pkg, quietly = TRUE)) {
    library(pkg, character.only = TRUE)
  } else {
    message(sprintf("Package %s is still not available after attempted installation.", pkg))
  }
}
pkgs <- c("lme4","readr","performance","dplyr","pwr", "parallel", "stringr")
lapply(pkgs, function(pkg){
  install_if_missing(pkg)
})
install_if_missing("parallelly", github = "futureverse/parallelly")
install_if_missing("broom.mixed", github = "bbolker/broom.mixed")
sessionInfo()

Loading required package: Matrix


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[[1]]
[1] "lme4"      "Matrix"    "stats"     "graphics"  "grDevices" "utils"    
[7] "datasets"  "methods"   "base"     

[[2]]
 [1] "readr"     "lme4"      "Matrix"    "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "performance" "readr"       "lme4"        "Matrix"      "stats"      
 [6] "graphics"    "grDevices"   "utils"       "datasets"    "methods"    
[11] "base"       

[[4]]
 [1] "dplyr"       "performance" "readr"       "lme4"        "Matrix"     
 [6] "stats"       "graphics"    "grDevices"   "utils"       "datasets"   
[11] "methods"     "base"       

[[5]]
 [1] "pwr"         "dplyr"       "performance" "readr"       "lme4"       
 [6] "Matrix"      "stats"       "graphics"    "grDevices"   "utils"      
[11] "datasets"    "methods"     "base"       

[[6]]
 [1] "parallel"    "pwr"         "dplyr"       "performance" "readr"      
 [6] "lme4"        "Matrix"      "stats"       "graphics"    "grDevices"  
[11] "utils"       "datasets"    "methods"     "base"       

[[7]]
 [1] "stringr"     "parallel"    "pwr"         "dplyr"       "performance"
 [6] "readr"       "lme4"        "Matrix"      "stats"       "graphics"   
[11] "grDevices"   "utils"       "datasets"    "methods"     "base"

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20.0.0 (64-bit)
Running under: macOS 15.5

Matrix products: default
BLAS/LAPACK: /Users/zory/miniforge3/envs/py310/lib/libopenblas.0.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: system (macOS)

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] broom.mixed_0.2.9.7 parallelly_1.44.0   stringr_1.5.1      
[4] pwr_1.3-0           dplyr_1.1.4         performance_0.14.0 
[7] readr_2.1.5         lme4_1.1-35.5       Matrix_1.6-5       

loaded via a namespace (and not attached):
 [1] future_1.49.0     generics_0.1.4    tidyr_1.3.1       stringi_1.8.7    
 [5] lattice_0.22-6    listenv_0.9.1     hms_1.1.3         digest_0.6.37    
 [9] magrittr_2.0.3    evaluate_0.22     grid_4.3.3        pbdZMQ_0.3-14    
[13] fastmap_1.2.0     jsonlite_2.0.0    backports_1.5.0   purrr_1.0.4      
[17] codetools_0.2

In [2]:
df <- read_csv("result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(c('front', 'left', 'right')),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df %>% mutate(
  offset = log(1/n_candidates / (1 - 1/n_candidates))
)

GROUP_LIST <- c(
  "Humans",
  "glm-4v-9b",
  "gemini-1.5-pro",
  "gpt-4o",
  "Qwen2.5-VL-72B-Instruct",
  "internlm-xcomposer2-vl-7b"
)

api_to_name <- list(
  "Humans" = "Humans",
  "gemini-1.5-pro" = "Gemini",
  "gpt-4o" = "GPT",
  "Qwen2.5-VL-72B-Instruct" = "Qwen",
  "internlm-xcomposer2-vl-7b" = "InternLM",
  "glm-4v-9b" = "GLM"
)

n_stimuli <- 900
n_prompts <- 12  # for VLMs
n_reps <- 10     # 10 repetitions per stimulus for VLMs
n_participants <- 65  # for humans
n_trials <- 45  # 45 trials per participant for humans
total_trials_vlm <- n_stimuli * n_reps
total_trials_human <- n_participants * n_trials  # 65 participants × 45 trials

Rows: 156825 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
get_formula <- function(grp){
  re <- if (grp=="Humans") {
    "(1 | Stimulus_ID) + (1 | Participant_ID)"
  } else if (grp %in% c("gemini-1.5-pro","Qwen2.5-VL-72B-Instruct")) {
    "(1 | Stimulus_ID) + (1 | Prompt_ID)"
  } else {
    "(1 | Stimulus_ID)"
  }
  as.formula(paste0(
    "Accuracy ~ Angle + scale(Proximity, scale=FALSE)",
    " + scale(n_candidates, scale=FALSE) + Actor + offset(offset) + ",
    re
  ))
}

models <- list()
for (grp in GROUP_LIST) {
  models[[grp]] <- lme4::glmer(
    get_formula(grp),
    data = df %>% filter(Group==grp, list_id!="-1", Part!="p0"),
    family = binomial(link="logit")
  )
}

# Extract ICC

icc(Stimulus_ID) = expected correlation of two randomly drawn observations from the same stimulus

In [4]:
get_icc_performance <- function(models_list, name_map) {
  icc_list <- lapply(names(models_list), function(model_key) {
    model <- models_list[[model_key]]
    
    # Calculate ICC using performance::icc() with by_group = TRUE
    icc_output <- tryCatch({
      performance::icc(model, by_group = TRUE) # Get ICC for each random effect
    }, error = function(e) {
      message(sprintf("Error calculating ICC for model %s: %s", model_key, e$message))
      # Return a data frame with expected columns from by_group = TRUE output
      return(data.frame(Group = character(0), ICC = numeric(0))) 
    })
    
    # Add the descriptive model name and original group key
    # This will apply to all rows in icc_output (if any)
    # If icc_output has 0 rows, this will result in a 0-row df with the new columns
    # (which is standard dplyr behavior)
    icc_output <- icc_output %>%
         mutate(Model = name_map[[model_key]],
                Original_Group = model_key) 
    
    return(icc_output)
  })
  
  # Combine all ICC data frames
  bind_rows(icc_list)
}

vc2_df <- get_icc_performance(models, api_to_name) # Pass api_to_name here
vc2_df

Group,ICC,Model,Original_Group
<chr>,<dbl>,<chr>,<chr>
Stimulus_ID,0.426272885,Humans,Humans
Participant_ID,0.062713402,Humans,Humans
Stimulus_ID,0.884636603,GLM,glm-4v-9b
Stimulus_ID,0.831444885,Gemini,gemini-1.5-pro
Prompt_ID,0.001253011,Gemini,gemini-1.5-pro
Stimulus_ID,0.739731872,GPT,gpt-4o
Stimulus_ID,0.854037287,Qwen,Qwen2.5-VL-72B-Instruct
Prompt_ID,0.001569549,Qwen,Qwen2.5-VL-72B-Instruct
Stimulus_ID,0.901237990,InternLM,internlm-xcomposer2-vl-7b


# Analytic (approximation only)

In [5]:
calculate_effective_n <- function(total_n, cluster_size, icc) {
  # Design effect due to clustering
  design_effect <- 1 + (cluster_size - 1) * icc
  effective_n <- total_n / design_effect
  return(effective_n)
}

In [6]:
# Ensure vc2_df is available from previous cells, containing columns:
# 'Original_Group' (e.g., "Humans", "gpt-4o"), 'Group' (random effect, e.g., "Stimulus_ID"), 'ICC'

effective_n_results <- list()

if (!exists("vc2_df")) {
  stop("Error: vc2_df is not found. Please ensure it's created in a preceding cell.")
}
if (!exists("calculate_effective_n")) {
  stop("Error: calculate_effective_n function is not found. Please ensure it's defined.")
}

for (grp_key in GROUP_LIST) {
  model_name <- api_to_name[[grp_key]]
  
  # Get ICC for Stimulus_ID for this model
  icc_stimulus_row <- vc2_df %>% filter(Original_Group == grp_key, Group == "Stimulus_ID")
  
  if (nrow(icc_stimulus_row) == 0) {
    message(sprintf("Warning: ICC for Stimulus_ID not found for group %s (Model: %s). Skipping N_eff calculation.", grp_key, model_name))
    effective_n_results[[grp_key]] <- list(
      model_name = model_name,
      total_n = NA,
      icc_stimulus = NA,
      cluster_size_stimulus = NA,
      eff_n = NA,
      other_iccs = NULL,
      notes = "ICC for Stimulus_ID not found."
    )
    next
  }
  icc_S <- icc_stimulus_row$ICC[1] # Take the first if multiple (should be unique)
  
  total_n <- 0
  cluster_size_S <- 0
  
  if (grp_key == "Humans") {
    total_n <- total_trials_human
    cluster_size_S <- total_trials_human / n_stimuli # Avg trials per stimulus for humans
  } else { # VLMs
    total_n <- total_trials_vlm
    cluster_size_S <- n_reps # Repetitions per stimulus for VLMs
  }
  
  eff_n <- calculate_effective_n(total_n, cluster_size_S, icc_S)
  
  # Store other ICCs for context
  other_iccs_df <- vc2_df %>% filter(Original_Group == grp_key, Group != "Stimulus_ID")
  other_iccs_list <- if(nrow(other_iccs_df) > 0) setNames(as.list(other_iccs_df$ICC), other_iccs_df$Group) else list()
  
  effective_n_results[[grp_key]] <- list(
    model_name = model_name,
    total_n = total_n,
    icc_stimulus = icc_S,
    cluster_size_stimulus = cluster_size_S,
    eff_n = eff_n,
    other_iccs = other_iccs_list
  )
}

# Print effective sample sizes
cat("Effective sample sizes (adjusted primarily for Stimulus_ID clustering):\n")
for (grp_key in GROUP_LIST) {
  result <- effective_n_results[[grp_key]]
  cat(sprintf("Model: %s\n", result$model_name))
  cat(sprintf("  Total N: %d\n", result$total_n))
  cat(sprintf("  ICC (Stimulus_ID): %.4f\n", result$icc_stimulus))
  cat(sprintf("  Cluster Size (Stimulus_ID): %.2f\n", result$cluster_size_stimulus))
  cat(sprintf("  Effective N: %.2f\n", result$eff_n))
  
  if (length(result$other_iccs) > 0) {
    cat("  Other ICCs:\n")
    for (group in names(result$other_iccs)) {
      cat(sprintf("    %s: %.4f\n", group, result$other_iccs[[group]]))
    }
  } else {
    cat("  No other ICCs available.\n")
  }
  
  cat("\n")
}

Effective sample sizes (adjusted primarily for Stimulus_ID clustering):
Model: Humans
  Total N: 2925
  ICC (Stimulus_ID): 0.4263
  Cluster Size (Stimulus_ID): 3.25
  Effective N: 1493.02
  Other ICCs:
    Participant_ID: 0.0627

Model: GLM
  Total N: 9000
  ICC (Stimulus_ID): 0.8846
  Cluster Size (Stimulus_ID): 10.00
  Effective N: 1004.27
  No other ICCs available.

Model: Gemini
  Total N: 9000
  ICC (Stimulus_ID): 0.8314
  Cluster Size (Stimulus_ID): 10.00
  Effective N: 1060.94
  Other ICCs:
    Prompt_ID: 0.0013

Model: GPT
  Total N: 9000
  ICC (Stimulus_ID): 0.7397
  Cluster Size (Stimulus_ID): 10.00
  Effective N: 1175.30
  No other ICCs available.

Model: Qwen
  Total N: 9000
  ICC (Stimulus_ID): 0.8540
  Cluster Size (Stimulus_ID): 10.00
  Effective N: 1036.11
  Other ICCs:
    Prompt_ID: 0.0016

Model: InternLM
  Total N: 9000
  ICC (Stimulus_ID): 0.9012
  Cluster Size (Stimulus_ID): 10.00
  Effective N: 987.80
  No other ICCs available.



In [7]:
d_to_logit <- function(d) {
  # Convert Cohen's d back to logit coefficient
  return(d * pi / sqrt(3))
}

# Calculate power for different effect sizes
effect_sizes_d <- seq(0.05, 0.5, by = 0.01)  # Cohen's d scale
all_power_results <- list()

if (!exists("effective_n_results")) {
  stop("Error: 'effective_n_results' not found. Run the previous cell.")
}

for (grp_key in names(effective_n_results)) {
  res <- effective_n_results[[grp_key]]
  current_eff_n <- res$eff_n
  
  if (is.na(current_eff_n) || current_eff_n <= 0) {
      message(sprintf("Skipping power calculation for %s (%s) due to invalid N_eff (%.1f)", res$model_name, grp_key, current_eff_n))
      next
  }
  
  # n for pwr.t.test is sample size per group for a two-sample t-test.
  # We assume current_eff_n is the total effective N for the model/group,
  # and the comparison of interest (e.g., factor level) involves splitting this into two sub-groups.
  sample_size_per_comparison_group <- current_eff_n / 2
  
  power_values <- numeric(length(effect_sizes_d))
  if (sample_size_per_comparison_group < 2) { # pwr.t.test requires n >= 2
      message(sprintf("Effective sample size per comparison group for %s (%.1f) is < 2. Power set to 0.", res$model_name, sample_size_per_comparison_group))
      power_values <- rep(0, length(effect_sizes_d))
  } else {
      power_values <- sapply(effect_sizes_d, function(d) {
        tryCatch({
          pwr.t.test(n = sample_size_per_comparison_group, d = d, sig.level = 0.05, type = "two.sample")$power
        }, error = function(e) { 0 }) # Default to 0 power on error
      })
  }
  
  power_df_grp <- data.frame(
    Model = res$model_name,
    Original_Group = grp_key,
    Cohens_d = effect_sizes_d,
    Logit_coefficient = d_to_logit(effect_sizes_d),
    Power = power_values,
    N_eff_total = current_eff_n,
    N_per_comparison_group = sample_size_per_comparison_group
  )
  all_power_results[[grp_key]] <- power_df_grp
}

power_table_analytic <- do.call(rbind, all_power_results)
if (!is.null(power_table_analytic)) {
  rownames(power_table_analytic) <- NULL
}

print("Analytical Power Analysis Table (based on N_eff from Stimulus_ID ICC):")
head(power_table_analytic)

[1] "Analytical Power Analysis Table (based on N_eff from Stimulus_ID ICC):"


,Model,Original_Group,Cohens_d,Logit_coefficient,Power,N_eff_total,N_per_comparison_group
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Humans,Humans,0.05,0.09068997,0.1616871,1493.022,746.5109
2,Humans,Humans,0.06,0.10882796,0.2123241,1493.022,746.5109
3,Humans,Humans,0.07,0.12696596,0.2719094,1493.022,746.5109
4,Humans,Humans,0.08,0.14510395,0.3391628,1493.022,746.5109
5,Humans,Humans,0.09,0.16324194,0.4121489,1493.022,746.5109
6,Humans,Humans,0.10,0.18137994,0.4883920,1493.022,746.5109


In [8]:
# Summary table for minimum detectable effect sizes at 80% power
if (!is.null(power_table_analytic) && nrow(power_table_analytic) > 0) {
  min_detectable_effects_analytic <- power_table_analytic %>%
    filter(!is.na(Power)) %>%
    group_by(Model, Original_Group, N_eff_total, N_per_comparison_group) %>%
    filter(Power >= 0.80) %>%
    slice_min(order_by = Cohens_d, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(Model, Original_Group, N_eff_total, N_per_comparison_group, Min_Cohens_d = Cohens_d, Min_Logit_Coeff = Logit_coefficient, Power_at_Min_d = Power)
  
  print("Minimum Detectable Effect Sizes (80% Power, Analytic Approximation):")
  min_detectable_effects_analytic
} else {
  print("No models had sufficient data for MDE calculation at 80% power or power_table_analytic is empty.")
  min_detectable_effects_analytic <- data.frame() # Ensure it exists as empty df
}

[1] "Minimum Detectable Effect Sizes (80% Power, Analytic Approximation):"


Model,Original_Group,N_eff_total,N_per_comparison_group,Min_Cohens_d,Min_Logit_Coeff,Power_at_Min_d
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GLM,glm-4v-9b,1004.2704,502.1352,0.18,0.3264839,0.8131127
GPT,gpt-4o,1175.3050,587.6525,0.17,0.3083459,0.8293712
Gemini,gemini-1.5-pro,1060.9449,530.4725,0.18,0.3264839,0.8336954
Humans,Humans,1493.0219,746.5109,0.15,0.2720699,0.8254006
InternLM,internlm-xcomposer2-vl-7b,987.8015,493.9008,0.18,0.3264839,0.8067331
Qwen,Qwen2.5-VL-72B-Instruct,1036.1101,518.0550,0.18,0.3264839,0.8249323


In [9]:
all_tost_results <- list()
alpha_tost <- 0.05 # Significance level for equivalence testing

for (grp_key in names(models)) {
  model <- models[[grp_key]]
  model_name_descriptive <- api_to_name[[grp_key]]
  
  # Get model summary for fixed effects
  model_summary <- summary(model)$coefficients
  model_summary_df <- as.data.frame(model_summary)
  model_summary_df$Term <- rownames(model_summary_df)
  rownames(model_summary_df) <- NULL
  
  # Calculate denominator for standardizing coefficients (as you did before)
  vc_list <- lapply(lme4::VarCorr(model), function(x) attr(x, "stddev")^2)
  vc <- sum(unlist(vc_list), na.rm = TRUE) # Added na.rm = TRUE for robustness
  effect_size_denom_t <- sqrt(vc + (pi^2/3))
  
  # Get SESOI (Min_Cohens_d) for this group from the analytical power analysis
  mde_row <- min_detectable_effects_analytic %>%
    filter(Original_Group == grp_key)
  
  sesoi_d <- NA
  notes_for_group <- ""
  
  if (nrow(mde_row) == 0 || is.na(mde_row$Min_Cohens_d[1])) {
    message(sprintf("Warning: MDE/SESOI (Min_Cohens_d) not found for group %s (Model: %s). TOST p-values will be NA.", grp_key, model_name_descriptive))
    notes_for_group <- "MDE/SESOI not available"
  } else {
    sesoi_d <- mde_row$Min_Cohens_d[1]
  }
  
  group_tost_results_list <- list()
  
  for (i in 1:nrow(model_summary_df)) {
    term_name <- model_summary_df$Term[i]
    estimate_log_odds <- model_summary_df$Estimate[i]
    se_log_odds <- model_summary_df$`Std. Error`[i]
    original_p_value <- model_summary_df$`Pr(>|z|)`[i]

    # Standardize estimate and SE
    estimate_std <- estimate_log_odds / effect_size_denom_t
    se_std <- se_log_odds / effect_size_denom_t
    
    p_upper_tost <- NA
    p_lower_tost <- NA
    p_value_tost <- NA
    is_equivalent <- NA
    term_notes <- notes_for_group
    
    if (term_name == "(Intercept)") {
      term_notes <- paste(notes_for_group, "TOST not typically applied to intercept", sep="; ")
    } else if (is.na(sesoi_d)) {
      # Already handled by notes_for_group, is_equivalent remains NA
    } else if (is.na(estimate_std) || is.na(se_std) || se_std <= 0) {
        term_notes <- paste(notes_for_group, "Invalid estimate or SE for TOST", sep="; ")
    } else {
      # Equivalence bounds (symmetric around 0 on the standardized scale)
      # We are testing if the effect is within [-sesoi_d, +sesoi_d]
      equiv_bound_upper <- sesoi_d
      equiv_bound_lower <- -sesoi_d
      
      # Test 1: Is the effect significantly less than the upper equivalence bound?
      # H0: estimate_std >= equiv_bound_upper  vs. H1: estimate_std < equiv_bound_upper
      z_upper <- (estimate_std - equiv_bound_upper) / se_std
      p_upper_tost <- pnorm(z_upper, lower.tail = TRUE)
      
      # Test 2: Is the effect significantly greater than the lower equivalence bound?
      # H0: estimate_std <= equiv_bound_lower vs. H1: estimate_std > equiv_bound_lower
      z_lower <- (estimate_std - equiv_bound_lower) / se_std
      p_lower_tost <- pnorm(z_lower, lower.tail = FALSE)
      
      # For equivalence, both one-sided tests must be significant
      p_value_tost <- max(p_upper_tost, p_lower_tost)
      is_equivalent <- p_value_tost < alpha_tost
    }
    
    group_tost_results_list[[term_name]] <- data.frame(
      Model = model_name_descriptive,
      Original_Group = grp_key,
      Term = term_name,
      Estimate_log_odds = estimate_log_odds,
      Std_Error_log_odds = se_log_odds,
      Original_P_Value = original_p_value,
      Estimate_std = estimate_std, # Standardized effect size (Cohen's d like)
      Std_Error_std = se_std,
      SESOI_d = sesoi_d,
      P_Upper_TOST = p_upper_tost,
      P_Lower_TOST = p_lower_tost,
      P_Value_TOST = p_value_tost,
      Is_Equivalent_TOST = is_equivalent,
      Notes = term_notes
    )
  }
  if (length(group_tost_results_list) > 0) {
    all_tost_results[[grp_key]] <- do.call(rbind, group_tost_results_list)
  }
}

if (length(all_tost_results) > 0) {
  tost_summary_table <- do.call(rbind, all_tost_results)
  rownames(tost_summary_table) <- NULL
  tost_summary_table_interpreted <- tost_summary_table %>%
    filter(Term != "(Intercept)") %>%
    arrange(Term) %>%
    select(Model, Term, Estimate_log_odds, Original_P_Value, Is_Equivalent_TOST, Notes) %>%
    mutate(
      Notes = case_when(
        Original_P_Value > 0.05 & Is_Equivalent_TOST == TRUE ~ "true null",
        Original_P_Value > 0.05 & (Is_Equivalent_TOST == FALSE | is.na(Is_Equivalent_TOST)) ~ "underpowered",
        Original_P_Value <= 0.05 & Is_Equivalent_TOST == TRUE ~ "true but small",
        Original_P_Value <= 0.05 & (Is_Equivalent_TOST == FALSE | is.na(Is_Equivalent_TOST)) ~ "meaningful",
        TRUE ~ "check logic" # Default case, should not be reached if logic is complete
      )
    )
  print("TOST Equivalence Test Summary:")
  tost_summary_table_interpreted
} else {
  print("No TOST results generated. Check warnings.")
}

# Interpretation guide for TOST results (when original p-value is > 0.05, i.e., non-significant):
# 1. Original_P_Value > 0.05 AND Is_Equivalent_TOST == TRUE:
#    Suggests the effect is not statistically different from zero, AND it's statistically within your
#    predefined equivalence bounds (i.e., practically negligible or too small to be of interest,
#    as defined by your SESOI_d). This supports an interpretation of a "null" or "negligible" effect.
#
# 2. Original_P_Value > 0.05 AND Is_Equivalent_TOST == FALSE (or NA):
#    The effect is not statistically different from zero, BUT you cannot conclude it's within your
#    equivalence bounds. This is an inconclusive result regarding practical equivalence.
#    The true effect might be small, or your study might lack the precision (power) to demonstrate
#    that it falls within the narrow equivalence bounds. This aligns with the "lack of power / inconclusive" scenario.
#
# 3. Original_P_Value <= 0.05 AND Is_Equivalent_TOST == FALSE:
#    The effect is statistically different from zero AND it's not statistically within your equivalence bounds.
#    This suggests a "meaningful" effect (i.e., different from zero and not negligible).
#
# 4. Original_P_Value <= 0.05 AND Is_Equivalent_TOST == TRUE:
#    The effect is statistically different from zero BUT it's also statistically within your equivalence bounds.
#    This is less common but implies a precisely estimated effect that is very small (different from zero, but still negligible).

[1] "TOST Equivalence Test Summary:"


Model,Term,Estimate_log_odds,Original_P_Value,Is_Equivalent_TOST,Notes
<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>
Humans,ActorY,-1.0032615,2.554306e-05,FALSE,meaningful
GLM,ActorY,-1.3476203,1.253278e-03,FALSE,meaningful
Gemini,ActorY,-0.1659441,5.978654e-01,TRUE,true null
GPT,ActorY,-0.9381818,5.638863e-05,FALSE,meaningful
Qwen,ActorY,-0.7439780,3.320191e-02,FALSE,meaningful
InternLM,ActorY,-1.1438885,1.417229e-02,FALSE,meaningful
Humans,Angleleft,-3.2358288,1.678691e-14,FALSE,meaningful
GLM,Angleleft,-0.3161462,5.170334e-01,FALSE,underpowered
Gemini,Angleleft,-0.4442305,2.440094e-01,FALSE,underpowered


# Simulation-based

In [ ]:
# Revised simulate_power function
simulate_power <- function(df_template, model_template_arg, formula_to_fit, target_term, cohen_d, n_obs) {
  # Extract parameters from the template model for the current group
  vc_template <- lme4::VarCorr(model_template_arg)
  re_variances_template <- sapply(vc_template, function(v) v[1]) # Variances
  fixed_intercept_template <- tryCatch(fixef(model_template_arg)["(Intercept)"], error = function(e) 0)

  # Determine RE factor names directly from the model's VarCorr object
  re_factors_in_formula <- names(re_variances_template)
  # Remove any potential NA names if a variance component was NA (though sapply should handle this)
  re_factors_in_formula <- re_factors_in_formula[!is.na(re_factors_in_formula)]

  # Calculate total latent variance for Cohen's d to log-odds conversion
  active_re_variances_sum <- 0
  if (length(re_factors_in_formula) > 0) {
      # We use re_variances_template directly as it already contains variances for factors in the model
      active_re_variances_sum <- sum(re_variances_template[re_factors_in_formula], na.rm = TRUE)
  }
  total_latent_variance <- active_re_variances_sum + (pi^2 / 3)
  log_odds_effect <- cohen_d * sqrt(total_latent_variance)

  # Simulate predictors based on the empirical distribution in df_template
  sim_data <- data.frame(row_id = 1:n_obs)
  sim_data$Angle <- sample(df_template$Angle, n_obs, replace = TRUE)
  sim_data$Proximity_raw <- sample(df_template$Proximity, n_obs, replace = TRUE)
  sim_data$n_candidates_raw <- sample(df_template$n_candidates, n_obs, replace = TRUE)
  sim_data$Actor <- sample(df_template$Actor, n_obs, replace = TRUE)

  # Scale predictors as in the original model formula
  sim_data$Proximity <- scale(sim_data$Proximity_raw, center = TRUE, scale = FALSE)[,1]
  sim_data$n_candidates <- scale(sim_data$n_candidates_raw, center = TRUE, scale = FALSE)[,1]

  # Calculate offset
  sim_data$offset <- log(1/sim_data$n_candidates_raw / (1 - 1/sim_data$n_candidates_raw))
  sim_data$offset[is.infinite(sim_data$offset) | is.na(sim_data$offset)] <- 0 # Handle potential division by zero if n_candidates is 1 or 0

  # Simulate Random Effects
  total_re_contribution <- numeric(n_obs)
  if (length(re_factors_in_formula) > 0) {
    for (re_factor_name in re_factors_in_formula) {
      # Ensure the variance is positive and the factor exists in df_template
      if (re_factor_name %in% names(re_variances_template) && !is.na(re_variances_template[[re_factor_name]]) && re_variances_template[[re_factor_name]] > 0 && re_factor_name %in% names(df_template)) {
        original_levels <- unique(df_template[[re_factor_name]])
        if(length(original_levels) == 0 || all(is.na(original_levels))) { # Handle cases with no levels or all NA levels
            # message(sprintf("Warning: No valid levels for RE factor %s in df_template for group. Using a dummy factor.", re_factor_name))
            sim_data[[re_factor_name]] <- factor(rep(1, n_obs)) # Dummy factor
            # No RE contribution if no valid levels to sample from or variance is zero
            next 
        }
        sim_data[[re_factor_name]] <- factor(sample(original_levels, n_obs, replace = TRUE))
        
        n_levels_current_re <- nlevels(sim_data[[re_factor_name]])
        re_sd <- sqrt(re_variances_template[[re_factor_name]])
        population_effects <- rnorm(n_levels_current_re, 0, re_sd)
        names(population_effects) <- levels(sim_data[[re_factor_name]])
        # Ensure all sampled levels in sim_data[[re_factor_name]] are present in names(population_effects)
        # This can be an issue if sampling from original_levels leads to fewer unique levels than n_levels_current_re expects
        # A safer way is to map directly from the sampled factor levels
        current_sim_levels <- as.character(sim_data[[re_factor_name]])
        missing_levels_in_pop_effects <- setdiff(unique(current_sim_levels), names(population_effects))
        if(length(missing_levels_in_pop_effects) > 0){
            # This case should ideally not happen if population_effects are generated based on levels(sim_data[[re_factor_name]])
            # message(sprintf("Warning: Mismatch in RE levels for %s. Some simulated levels not in population_effects map.", re_factor_name))
            # For robustness, assign 0 to these missing levels, though it indicates a deeper issue if it occurs.
            temp_effects_for_missing <- rep(0, length(missing_levels_in_pop_effects))
            names(temp_effects_for_missing) <- missing_levels_in_pop_effects
            population_effects <- c(population_effects, temp_effects_for_missing)
        }
        total_re_contribution <- total_re_contribution + population_effects[current_sim_levels]
      } else { 
        # If RE not in model (VarCorr), variance is zero/NA, or factor not in df_template, create a dummy factor column if it doesn't exist
        if (!re_factor_name %in% names(sim_data)) {
            sim_data[[re_factor_name]] <- factor(rep(1, n_obs)) # Dummy factor
        }
        # No RE contribution added
      }
    }
  }

  # Calculate linear predictor (eta)
  eta <- fixed_intercept_template + sim_data$offset + total_re_contribution
  # Add the target effect (log_odds_effect) for the specific term being tested
  # Other fixed effects are assumed to be 0 for this specific power calculation
  if (target_term == "Angleleft") { eta <- eta + ifelse(sim_data$Angle == "left", log_odds_effect, 0) }
  else if (target_term == "Angleright") { eta <- eta + ifelse(sim_data$Angle == "right", log_odds_effect, 0) }
  else if (target_term == "scale(Proximity, scale = FALSE)") { eta <- eta + sim_data$Proximity * log_odds_effect }
  else if (target_term == "scale(n_candidates, scale = FALSE)") { eta <- eta + sim_data$n_candidates * log_odds_effect }
  else if (target_term == "ActorY") { eta <- eta + ifelse(sim_data$Actor == "Y", log_odds_effect, 0) }

  # Simulate binary outcome
  prob <- plogis(eta)
  sim_data$Accuracy <- rbinom(n_obs, 1, prob)

  # Fit model to simulated data
  p_value_target_term <- NA
  tryCatch({
    # Ensure the offset column in sim_data is named 'offset' as expected by get_formula
    model_fit_sim <- lme4::glmer(formula_to_fit, data = sim_data, family = binomial(link = "logit"))
    model_summary <- broom.mixed::tidy(model_fit_sim, effects = "fixed")
    term_row <- model_summary[model_summary$term == target_term, ]
    if (nrow(term_row) == 1) {
      p_value_target_term <- term_row$p.value
    }
  }, error = function(e) {
    # message(sprintf("Simulation glmer error for %s, term %s, d=%.2f: %s", deparse(substitute(model_template_arg)), target_term, cohen_d, e$message))
    p_value_target_term <- NA
  })
  return(list(p_value = p_value_target_term, significant = !is.na(p_value_target_term) && p_value_target_term < 0.05))
}

# Main simulation loop
TERMS_TO_TEST <- list(
  "Angleleft" = "Angleleft",
  "Angleright" = "Angleright",
  "Proximity_scaled" = "scale(Proximity, scale = FALSE)"
#  "n_candidates_scaled" = "scale(n_candidates, scale = FALSE)",
)

effect_sizes_cohen_d_sim <- seq(0.05, 0.4, by = 0.05) # Define range of Cohen's d
n_simulations <- 100 # Number of simulations per setting. 1 -> 20min; 20 -> 50min, 100 -> 3h
simulation_power_results_list <- list()

for (grp_key in GROUP_LIST) {
  message(sprintf("--- Processing Group: %s ---", grp_key))
  df_grp_template <- df %>% filter(Group == grp_key, list_id != "-1", Part != "p0")
  if (nrow(df_grp_template) == 0) {
    message(sprintf("  Skipping group %s: No data after filtering.", grp_key))
    next
  }
  model_template_for_grp <- models[[grp_key]]
  if (is.null(model_template_for_grp)) {
    message(sprintf("  Skipping group %s: Original model not found.", grp_key))
    next
  }
  formula_for_grp <- get_formula(grp_key)
  n_obs_for_sim <- nrow(df_grp_template)

  for (term_friendly_name in names(TERMS_TO_TEST)) {
    target_coefficient_name <- TERMS_TO_TEST[[term_friendly_name]]
    message(sprintf("  Testing Term: %s (Coefficient: %s)", term_friendly_name, target_coefficient_name))

    # Check if the target coefficient exists in the original model for this group
    original_coeffs <- tryCatch(rownames(summary(model_template_for_grp)$coefficients), error = function(e) character(0))
    if (!target_coefficient_name %in% original_coeffs) {
        # This can happen if a factor level is not present or reference level logic changes things
        # For Angleleft/Angleright, if only one non-reference level exists, the other won't be in summary
        # For now, we'll attempt simulation; if term is truly absent, p-value will be NA from simulate_power
        message(sprintf("    Warning: Target coefficient '%s' not found in original model for group %s. Simulation will proceed but p-value might be NA.", target_coefficient_name, grp_key))
    }

    for (es_d in effect_sizes_cohen_d_sim) {
    #  message(sprintf("    Cohen's d: %.3f", es_d))
      num_cores <- max(1, parallel::detectCores() - 1)
      sim_outputs <- parallel::mclapply(1:n_simulations, function(sim_idx) {
        simulate_power(
          df_template = df_grp_template,
          model_template_arg = model_template_for_grp,
          formula_to_fit = formula_for_grp,
          target_term = target_coefficient_name,
          cohen_d = es_d,
          n_obs = n_obs_for_sim
        )
      }, mc.cores = num_cores)
      
      p_values_from_sims <- sapply(sim_outputs, function(x) if(is.list(x)) x$p_value else NA)
      successful_sims <- sum(!is.na(p_values_from_sims))
      power_estimate <- if (successful_sims > 0) {
        sum(p_values_from_sims < 0.05, na.rm = TRUE) / successful_sims
      } else { NA }
      
      simulation_power_results_list[[length(simulation_power_results_list) + 1]] <- data.frame(
        Model = api_to_name[[grp_key]],
        Original_Group = grp_key,
        Term_Friendly = term_friendly_name,
        Term_Coefficient = target_coefficient_name,
        Cohens_d = es_d,
        Power = power_estimate,
        N_sim_successful = successful_sims,
        N_sim_total = n_simulations
      )
    }
  }
}
final_power_simulation <- do.call(rbind, simulation_power_results_list)
if (!is.null(final_power_simulation)) rownames(final_power_simulation) <- NULL


--- Processing Group: Humans ---

  Testing Term: Angleleft (Coefficient: Angleleft)

  Testing Term: Angleright (Coefficient: Angleright)

  Testing Term: Proximity_scaled (Coefficient: scale(Proximity, scale = FALSE))

--- Processing Group: glm-4v-9b ---

  Testing Term: Angleleft (Coefficient: Angleleft)

  Testing Term: Angleright (Coefficient: Angleright)

  Testing Term: Proximity_scaled (Coefficient: scale(Proximity, scale = FALSE))

--- Processing Group: gemini-1.5-pro ---

  Testing Term: Angleleft (Coefficient: Angleleft)

  Testing Term: Angleright (Coefficient: Angleright)

  Testing Term: Proximity_scaled (Coefficient: scale(Proximity, scale = FALSE))

--- Processing Group: gpt-4o ---

  Testing Term: Angleleft (Coefficient: Angleleft)

  Testing Term: Angleright (Coefficient: Angleright)

  Testing Term: Proximity_scaled (Coefficient: scale(Proximity, scale = FALSE))

--- Processing Group: Qwen2.5-VL-72B-Instruct ---

  Testing Term: Angleleft (Coefficient: Angleleft)

  

In [14]:
print("Simulation-Based Power Analysis Results:")
head(final_power_simulation)

[1] "Simulation-Based Power Analysis Results:"


,Model,Original_Group,Term_Friendly,Term_Coefficient,Cohens_d,Power,N_sim_successful,N_sim_total
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<dbl>
1,Humans,Humans,Angleleft,Angleleft,0.05,0.02941176,34,100
2,Humans,Humans,Angleleft,Angleleft,0.10,0.08000000,25,100
3,Humans,Humans,Angleleft,Angleleft,0.15,0.00000000,22,100
4,Humans,Humans,Angleleft,Angleleft,0.20,0.02564103,39,100
5,Humans,Humans,Angleleft,Angleleft,0.25,0.16666667,30,100
6,Humans,Humans,Angleleft,Angleleft,0.30,0.06666667,30,100


In [12]:
# Minimum detectable effect size for each variable from simulation
if (exists("final_power_simulation") && nrow(final_power_simulation) > 0) {
  print("\nMinimum detectable effect sizes (80% power, Simulation-Based):")
  mdes_simulation <- final_power_simulation %>%
    filter(!is.na(Power) & Power >= 0.80) %>%
    group_by(Model, Original_Group, Term_Friendly, Term_Coefficient) %>%
    slice_min(order_by = Cohens_d, n = 1, with_ties = FALSE) %>%
    ungroup() %>%
    select(Model, Original_Group, Term_Friendly, Min_Cohens_d = Cohens_d, Power_at_Min_d = Power)
  mdes_simulation
} else {
  print("final_power_simulation table is empty or not found. Run the simulation cell.")
}

[1] "\nMinimum detectable effect sizes (80% power, Simulation-Based):"


Model,Original_Group,Term_Friendly,Min_Cohens_d,Power_at_Min_d
<chr>,<chr>,<chr>,<dbl>,<dbl>
GLM,glm-4v-9b,Angleleft,0.10,1.0000000
GLM,glm-4v-9b,Angleright,0.05,0.8265306
GLM,glm-4v-9b,Proximity_scaled,0.05,1.0000000
GPT,gpt-4o,Angleleft,0.10,1.0000000
GPT,gpt-4o,Angleright,0.10,1.0000000
GPT,gpt-4o,Proximity_scaled,0.05,0.9800000
Gemini,gemini-1.5-pro,Angleleft,0.10,1.0000000
Gemini,gemini-1.5-pro,Angleright,0.10,1.0000000
Gemini,gemini-1.5-pro,Proximity_scaled,0.05,1.0000000


In [13]:
simulation_all_tost_results <- list()
for (grp_key in names(models)) {
  model <- models[[grp_key]]
  model_name_descriptive <- api_to_name[[grp_key]]
  
  # Get model summary for fixed effects
  model_summary <- summary(model)$coefficients
  model_summary_df <- as.data.frame(model_summary)
  model_summary_df$Term <- rownames(model_summary_df)
  rownames(model_summary_df) <- NULL
  
  # Calculate denominator for standardizing coefficients (as you did before)
  vc_list <- lapply(lme4::VarCorr(model), function(x) attr(x, "stddev")^2)
  vc <- sum(unlist(vc_list), na.rm = TRUE) # Added na.rm = TRUE for robustness
  effect_size_denom_t <- sqrt(vc + (pi^2/3))
  
  # Get SESOI (Min_Cohens_d) for this group from the analytical power analysis
  mde_row <- mdes_simulation %>%
    filter(Original_Group == grp_key)
  
  sesoi_d <- NA
  notes_for_group <- ""
  
  if (nrow(mde_row) == 0 || is.na(mde_row$Min_Cohens_d[1])) {
    message(sprintf("Warning: MDE/SESOI (Min_Cohens_d) not found for group %s (Model: %s). TOST p-values will be NA.", grp_key, model_name_descriptive))
    notes_for_group <- "MDE/SESOI not available"
  } else {
    sesoi_d <- mde_row$Min_Cohens_d[1]
  }
  
  group_tost_results_list <- list()
  
  for (i in 1:nrow(model_summary_df)) {
    term_name <- model_summary_df$Term[i]
    estimate_log_odds <- model_summary_df$Estimate[i]
    se_log_odds <- model_summary_df$`Std. Error`[i]
    original_p_value <- model_summary_df$`Pr(>|z|)`[i]

    # Standardize estimate and SE
    estimate_std <- estimate_log_odds / effect_size_denom_t
    se_std <- se_log_odds / effect_size_denom_t
    
    p_upper_tost <- NA
    p_lower_tost <- NA
    p_value_tost <- NA
    is_equivalent <- NA
    term_notes <- notes_for_group
    
    if (term_name == "(Intercept)") {
      term_notes <- paste(notes_for_group, "TOST not typically applied to intercept", sep="; ")
    } else if (is.na(sesoi_d)) {
      # Already handled by notes_for_group, is_equivalent remains NA
    } else if (is.na(estimate_std) || is.na(se_std) || se_std <= 0) {
        term_notes <- paste(notes_for_group, "Invalid estimate or SE for TOST", sep="; ")
    } else {
      # Equivalence bounds (symmetric around 0 on the standardized scale)
      # We are testing if the effect is within [-sesoi_d, +sesoi_d]
      equiv_bound_upper <- sesoi_d
      equiv_bound_lower <- -sesoi_d
      
      # Test 1: Is the effect significantly less than the upper equivalence bound?
      # H0: estimate_std >= equiv_bound_upper  vs. H1: estimate_std < equiv_bound_upper
      z_upper <- (estimate_std - equiv_bound_upper) / se_std
      p_upper_tost <- pnorm(z_upper, lower.tail = TRUE)
      
      # Test 2: Is the effect significantly greater than the lower equivalence bound?
      # H0: estimate_std <= equiv_bound_lower vs. H1: estimate_std > equiv_bound_lower
      z_lower <- (estimate_std - equiv_bound_lower) / se_std
      p_lower_tost <- pnorm(z_lower, lower.tail = FALSE)
      
      # For equivalence, both one-sided tests must be significant
      p_value_tost <- max(p_upper_tost, p_lower_tost)
      is_equivalent <- p_value_tost < alpha_tost
    }
    
    group_tost_results_list[[term_name]] <- data.frame(
      Model = model_name_descriptive,
      Original_Group = grp_key,
      Term = term_name,
      Estimate_log_odds = estimate_log_odds,
      Std_Error_log_odds = se_log_odds,
      Original_P_Value = original_p_value,
      Estimate_std = estimate_std, # Standardized effect size (Cohen's d like)
      Std_Error_std = se_std,
      SESOI_d = sesoi_d,
      P_Upper_TOST = p_upper_tost,
      P_Lower_TOST = p_lower_tost,
      P_Value_TOST = p_value_tost,
      Is_Equivalent_TOST = is_equivalent,
      Notes = term_notes
    )
  }
  if (length(group_tost_results_list) > 0) {
    simulation_all_tost_results[[grp_key]] <- do.call(rbind, group_tost_results_list)
  }
}

if (length(simulation_all_tost_results) > 0) {
  simulation_tost_summary_table <- do.call(rbind, simulation_all_tost_results)
  rownames(simulation_tost_summary_table) <- NULL
  simulation_tost_summary_table_interpreted <- simulation_tost_summary_table %>%
    filter(Term != "(Intercept)") %>%
    arrange(Term) %>%
    select(Model, Term, Estimate_log_odds, Original_P_Value, Is_Equivalent_TOST, Notes) %>%
    mutate(
      Notes = case_when(
        Original_P_Value > 0.05 & Is_Equivalent_TOST == TRUE ~ "true null",
        Original_P_Value > 0.05 & (Is_Equivalent_TOST == FALSE | is.na(Is_Equivalent_TOST)) ~ "underpowered",
        Original_P_Value <= 0.05 & Is_Equivalent_TOST == TRUE ~ "true but small",
        Original_P_Value <= 0.05 & (Is_Equivalent_TOST == FALSE | is.na(Is_Equivalent_TOST)) ~ "meaningful",
        TRUE ~ "check logic" # Default case, should not be reached if logic is complete
      )
    )
  print("TOST Equivalence Test Summary:")
  simulation_tost_summary_table_interpreted
} else {
  print("No TOST results generated. Check warnings.")
}

[1] "TOST Equivalence Test Summary:"


Model,Term,Estimate_log_odds,Original_P_Value,Is_Equivalent_TOST,Notes
<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>
Humans,ActorY,-1.0032615,2.554306e-05,NA,meaningful
GLM,ActorY,-1.3476203,1.253278e-03,FALSE,meaningful
Gemini,ActorY,-0.1659441,5.978654e-01,FALSE,underpowered
GPT,ActorY,-0.9381818,5.638863e-05,FALSE,meaningful
Qwen,ActorY,-0.7439780,3.320191e-02,FALSE,meaningful
InternLM,ActorY,-1.1438885,1.417229e-02,FALSE,meaningful
Humans,Angleleft,-3.2358288,1.678691e-14,NA,meaningful
GLM,Angleleft,-0.3161462,5.170334e-01,FALSE,underpowered
Gemini,Angleleft,-0.4442305,2.440094e-01,FALSE,underpowered
